<a href="https://colab.research.google.com/github/0Nguyen0Cong0Tuan0/Road-Buddy-Challenge/blob/main/models/yolo_finetune_bdd100k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **YOLO11 Fine-tuning on BDD100K Dataset**

This notebook **fine-tunes YOLO11n and YOLO11l** on the BDD100K dataset for traffic object detection.

**Dataset**

| Dataset | Classes | Focus |
|---------|---------|-------|
| BDD100K | 12 | Vehicles, pedestrians, traffic signs/lights |


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **Setup & Installation**

In [ ]:
!pip install ultralytics -q

In [ ]:
import ultralytics
import os
import yaml
import time
from collections import Counter
import numpy as np
from pathlib import Path
from ultralytics import YOLO
import shutil
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'models' else Path.cwd()
DATA_DIR = PROJECT_ROOT / 'drive' / 'MyDrive' / 'traffic datasets'
MODELS_DIR = PROJECT_ROOT / 'drive' / 'MyDrive' / 'models'
RUNS_DIR = PROJECT_ROOT / 'drive' / 'MyDrive' / 'runs'

# Dataset paths
DATASETS = {
    'bdd100k': {
        'path': DATA_DIR / 'bdd100k',
        'yaml': DATA_DIR / 'bdd100k' / 'data_yolo.yaml',
        'description': 'BDD100K - Vehicles, pedestrians, traffic objects',
        'classes': ['car', 'truck', 'bus', 'train', 'person', 'rider',
                   'bike', 'motor', 'traffic light', 'traffic sign', 'lane', 'drivable area']
    }
}

for name, info in DATASETS.items():
    exists = info['path'].exists()
    yaml_exists = info['yaml'].exists()
    print(f"{name}: {info['path'].name}")
    print(f"   YAML: {info['yaml'].name} ({'found' if yaml_exists else 'NOT FOUND'})")
    print(f"   Classes: {len(info['classes'])} - {info['classes'][:5]}...")

In [ ]:
# Training configuration
TRAINING_CONFIG = {
    'epochs': 5,           # Number of training epochs
    'imgsz': 640,          # Image size
    'batch': 16,           # Batch size (reduce if GPU OOM)
    'patience': 10,        # Early stopping patience
    'device': DEVICE,      # Training device
    'workers': 4,          # Data loader workers
    'save': True,          # Save checkpoints
    'plots': True,         # Generate training plots
    'verbose': True,       # Verbose output
}


print("\nTraining Configuration")

for key, value in TRAINING_CONFIG.items():
    print(f"   {key}: {value}")

## **Fine-tuning on BDD100K Dataset**

In [ ]:
bdd_labels_dir = DATASETS['bdd100k']['path'] / 'yolo_labels'

if bdd_labels_dir.exists():
    train_labels = list((bdd_labels_dir / 'train').glob('*.txt')) if (bdd_labels_dir / 'train').exists() else []
    val_labels = list((bdd_labels_dir / 'val').glob('*.txt')) if (bdd_labels_dir / 'val').exists() else []
    print(f"BDD100K YOLO labels found:")
    print(f"   Train labels: {len(train_labels)}")
    print(f"   Val labels: {len(val_labels)}")
else:
    print("BDD100K YOLO labels not found!")

In [ ]:
# Fine-tune YOLO11n on BDD100K
model_n_bdd = YOLO('yolo11n.pt')

results_n_bdd = model_n_bdd.train(
    data=str(DATASETS['bdd100k']['yaml']),
    project=str(RUNS_DIR / 'finetune'),
    name='yolo11n_bdd100k',
    exist_ok=True,
    **TRAINING_CONFIG
)

In [ ]:
model_l_bdd = YOLO('yolo11l.pt')

config_l = TRAINING_CONFIG.copy()
config_l['batch'] = 8

results_l_bdd = model_l_bdd.train(
    data=str(DATASETS['bdd100k']['yaml']),
    project=str(RUNS_DIR / 'finetune'),
    name='yolo11l_bdd100k',
    exist_ok=True,
    **config_l
)